**Program Instructions:**


1.   Mount your drive: open file directory and click the drive icon to mount drive
2.   Run [2] and [3] to set up program dependencies
3.   Run [4] to load in the model
4. Run [5]: Position your hand gesture in the frame. Click Capture. Observe the model's prediction. Repeat as much as you want



In [ ]:
MODEL = '/content/drive/MyDrive/1770FinalProject/model'

**Config Image Capture**

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import io
import PIL
from keras.applications.mobilenet_v2 import preprocess_input


def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      video.width = 224; // Set the width of the video element to 224 pixels
      video.height = 224; // Set the height of the video element to 224 pixels
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = 224; // Set the width to 224
      canvas.height = 224; // Set the height to 224
      canvas.getContext('2d').drawImage(video, 0, 0, 224, 224); // Resize the image to 224x224

      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))


  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)

  image = PIL.Image.open(io.BytesIO(binary))
  image = image.resize((224,224))
  image_array = np.array(image)
  image_array = preprocess_input(image_array)

  return filename, image_array

**Load the Model**

In [ ]:
from keras.models import load_model
import os

gesture_model = load_model(os.path.join(MODEL, 'trained_model.keras'))

**Run The Model**

In [ ]:
import os
from keras.models import load_model
import numpy as np

try:
  filename, image_array = take_photo()
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))


image_array = np.expand_dims(image_array, axis = 0)

prediction = gesture_model.predict(image_array)
# reconstructed_prediction = reconstructed_model.predict(image_array)

print(f"Prediction Matrix:{prediction}")

actual_pred = np.argmax(prediction, axis=1)

if actual_pred == 0:
  print("Prediction: thumbs down")
elif actual_pred == 1:
  print("Prediction: thumbs up")
elif actual_pred == 2:
  print("Prediction: number 1")
elif actual_pred == 3:
  print("Prediction: number 2")
elif actual_pred == 4:
  print("Prediction: number 3")
elif actual_pred == 5:
  print("Prediction: number 4")
elif actual_pred == 6:
  print("Prediction: number 5")

print(f"Confidence: {prediction[0,actual_pred]*100}%")
